### 1. 사전 검증

- 변수 확인
  - 독립변수와 종속변수 확인하기
    - sqft_living, sqft_lot는 상관관계가 높을 수 있겠구나 싶음
    - sqft_above+sqft_basement=sqft_living의 관계도 알아볼 수 있음
    - 그 외에도 독립변수들 간의 관계성을 도메인 지식을 통해 알아볼 수 있다.

- 기본적으로 확인할 것
1. 결측치 확인 -> df.isnull().sum()
2. 각 변수의 data type 확인 -> df.info()
3. 기본적인 통계 기술값 확인 -> df.describe()  
   -> 과제에서는 추가적으로 median, skew, kurtosis, null값까지 확인

종속변수(타겟)의 왜도가 치우쳐진것을 확인함  
-> 시각화(hist)를 통해서도 확인  
-> 자연로그 취해서 정규분포 형태로 만들어주기

함수를 통해 object, int, float 타입을 분류해서 데이터 살펴봄  
-> 확인 결과 data변수가 object형, 전처리 필요

-> date 컬럼에서 필요한 부분 astype하고 새로운 컬럼에 저장

도메인 지식에 따라 기존의 변수들을 활용하여 새로운 변수를 만들어낼 수도 있음.  
ex. sold-built_years = date2 - yr_built

lambda & apply 잘 활용하기  
lambda로 하고 싶은 처리 수행 하고 apply로 반복문 안쓰고 일괄 처리

- 전처리 이후 독립변수의 분포 및 종속변수와의 선형관계 파악 (시각화)
  - 분포 -> histogram으로 파악
  - 선형 관계 -> 변수 선정 이후 pairplot으로 파악
    -> price와 sqft_living간의 선형관계 볼 수 있음

- heatmap을 통해 독립변수와 종속변수의 상관관계, 독립변수 간의 상관관계도 시각화 해보자

In [4]:
'''
mask = np.zeros_like(df_corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True

-> heatmap의 대각선 위쪽 절반을 가릴 수 있음'''

'\nmask = np.zeros_like(df_corr, dtype=bool)\nmask[np.triu_indices_from(mask)] = True\n\n-> heatmap의 대각선 위쪽 절반을 가릴 수 있음'

이후, price와 상관관계가 높은 변수들을 asceding=Flase를 통해 추려서 확인해볼 수 있음  
-> 상관관계가 높은 세 변수를 찾아볼 수 있었다.

- 선택한 독립 변수들의 분포를 살펴보기  
-> 세 변수들의 분포가 비슷하네?  
-> 만약 다중공선성이 존재한다면, price와 가장 상관관계가 높은 sqft_living으로 분석해야겠다  
-> 독립변수 1개가 되므로 단순 선형 회귀 분석으로 할 생각

- 모델링 하기 전에 독립변수와 종속변수의 선형 관계 확인
  - jointplot사용
    - 산점도&회귀선 + 커널밀도함수로 분포까지 같이 볼 수 있음

### 2. 모델 생성 및 fit

- '항상 상수항 추가하기!'  
X=sm.add_constant(X,has_constant='add')

상수항 y값인데 왜 독립변수인 X에 추가하는지 궁금해서 찾아봄
-> 벡터 계산이므로 독립변수 벡터에 모든 값이 1인 열을 추가해서 계산 -> 상수항이 추가됨

- 모델 fit
  - statsmodel에서 OLS import하고 fit 시키기
  - 이후 summary를 통해 결과 확인
    - 확인 결과 p값이 유의수준보다 훨씬 낮음, 귀무가설 기각 가능 & 모델의 통계적 유의성 확보

- 잔차 확인  
result_model.resid.plot()으로 잔차 확인 가능  
시각화는 또또 plt.show()

### 2. 다중 선형 회귀

- 연속형 반응변수 하나에 설명변수가 둘 이상
- 다중공선성과 차원의 저주 문제가 발생할 수 있음

id를 제외한 모든 독립변수를 사용해 모델을 만들어 본 후, 독립변수를 선별해서 다시 선형 회귀 분석 실시.  
다 때려보고 그 중 괜찮은 것을 고르는 것이 좋다고 함 ~

list(df.columns)를 columns 변수에 저장하고, 쓰지 않을 column을 not_used 변수에 저장  
이후 반복문으로 쓰지 않을 column들을 remove하고  
columns만 선택한 df 선언해서 df_reg에 저장

df_reg에 상수항 추가해서 df_kc_reg에 저장  
feature_columns에 df_kc_reg에서 뽑아낸 price를 제외한 변수들을 리스트로 저장  
이때, df_kc_reg.columns.difference(['price'])를 통해 price를 뺀다.  

difference: 차집합 계산, set에서만 사용 가능하지만, index 객체도 set처럼 계산 가능해서 위처럼 사용 가능

이후 X,y에 독립변수, 종속변수 넣고 모델링 및 결과 확인하기

결과를 보니 p값도 0에 가깝고, R-squared값도 0.701로 잘 만든 모델처럼 보임  
but, 다중공선성 확인하기 (오버피팅 가능성)

확인 결과 다중공선성 문제가 발생하는 것을 확인할 수 있음  

VIF=inf(무한대)가 나오는 경우:  
VIF=1/(1-R^2)인데, 변수 X_i가 나머지 변수로 회귀했을때의 결정계수가 1인 경우 분모가 0이 되어(0에 수렴?) VIF가 inf가 되버림.

'bedrooms','sqft_living','waterfront','view','sold-built_years','price'만 독립변수로 선택해서 모델링하기

마찬가지로 df_reg에 독립변수들과 종속변수 넣고, 상수항들 추가해주고 price를 빼주기   
-> 근데 여기서 df_reg를 만들고 거기에 add.constant를 하면 price에도 상수항 계산하는게 아닌가? 했는데 알고보니 add.constant하면 데이터프레임에 상수열을 추가하는거고, 그 데이터프레임에서 price를 제외시키므로 종속변수에 상수항이 추가될 일은 없음

결과를 확인해보니 p값이 유의수준보다 낮아 통계적 유의성 확인 가능  
이후 다중공선성을 확인해보자  
-> 다중공선성이 줄어듬을 확인할 수 있음

### 회귀모형의 가정 진단

선형성, 정규성, 독립성, 오차항의 자기상관, 등분산성, 오차항의 평균==0인가?를 검정해준다.

1. 선형성 검정

- scatter로 검정
- fitted변수에 result_model_2.fittedvalues를 저장
  -> 모델을 통해 예측된 값들을 fitted에 저장

- fitted의 최소/최대값을 plot의 x,y값에 넣어 대각선을 그림

시각화 결과 y=x를 따라 어느정도 우상향하는 모습을 보임  
또한, 데이터가 심하게 곡선으로 휘는 형태를 보이지는 않으므로 선형성을 만족한다고 할 수 있음

2. 정규성 검정

- 모델의 잔차를 qqplot에 넣어 모델의 정규성을 검정해본다.
  - - result_model_2.resid: 모델의 잔차

- 잔차를 히스토그램(+kde)으로도 그려봐 정규분포 모양인지 확인해본다.

- 잔차와 예측값을 구해 예측값을 x축, 잔차를 y축에 놓고 scatter를 그려 잔차 패턴을 확인해본다
- fitted=result_model_2.predict()나, pred=result_model_2.predict(X)나 둘 다 예측값을 나타낸다
  - 보통 predict()를 하면 내부적으로 모델이 기억하고 있는 X값을 자동으로 사용해서 같다고 한다.

- 사피로 - 월크 검정
  - '데이터는 정규분포를 따른다'를 귀무가설로 설정하고 p-value계산

- 계산 결과가 0.05보다 작음 -> 정규성 없다고 해야하나?
- but, 관측치가 매우 높아 p값이 낮아질 수도 있음. qq플롯을 보자

3. 독립성 검정

다중공선성 확인했으니 pass

4. 오차항의 자기상관 검정

- ACF와 Durbin-Watson으로 독립성 만족 여부를 확인할 수 있음
- Durbin-Watson이 2에 가까우면 자기상관이 없다고 할 수 있음
- 결과가 1.956이므로 자기상관이 없다

5. 등분산성

- displot과 resid의 선형그래프(plot)를 통해 잔차의 등분산성 확인
- 대부분 -1~1사이에 있으므로 등분산성 만족!

6. 오차항의 평균이 0인가?

- np.mean(result_model_2.resid)로 평균 확인
- 0에 수렴하므로 만족한다고 할 수 있다.

기본 가정 6가지를 다 만족  
그러므로 이 모델은 적합하다고 할 수 있다 ~